In [1]:
from functions4 import *
import matplotlib.colors as colors
from scipy.integrate import dblquad

In [3]:
GAMMA,Phi=get_best_fit_values()
DETECTOR=LoadDETECTOR("1111")
localPath="/net/scratch_icecube4/user/glauch/"
datapath     = localPath+"DATA/"
SAVE_DIRECTORY = "/net/scratch_icecube4/user/kalaczynski/Plots/"

In [4]:
Aeff_res=plot_aeff_vs_dec_energy(simulationLoad(DETECTOR[1], datapath, 2.13)[0], logE_range=(2,9), sinDec_range=(np.sin(np.radians(-6)),1), bins=[40,40], title=None, savepath=None)

/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_11 the total uptime is 29556432.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2011_10602_11077_11191_corrected_ow.npy
EBinWidth 1384260 MC  length 1384260


In [5]:
X,Y=np.meshgrid(Aeff_res[2],Aeff_res[1])
H=Aeff_res[0]

In [6]:
Aeff= np.sum(H, axis=1)*(X[0][1]-X[0][0])
index=2.13 # assumed spectral index:
deltaE=np.array([10**Aeff_res[1][i+1]-10**Aeff_res[1][i] for i in range(len(Aeff_res[2])-1)])
E=np.array([(10**Aeff_res[1][i+1]+10**Aeff_res[1][i])/2 for i in range(len(Aeff_res[2])-1)])
Aeff_theta=[np.sum((E**(-index))*np.concatenate(H[:,i:i+1])*deltaE)/np.sum((E**(-index))*deltaE) for i in range(len(X)-1)]
E_avg=np.average(E)

In [10]:
print E_avg, "GeV"

62922635.8362 GeV


In [8]:
from matplotlib.ticker import NullFormatter
import copy as copy
# definitions for the axes
nullfmt = NullFormatter() 
left, width = 0.1, 0.6
bottom, height = 0.15, 0.55
bottom_h = left_h = left + width + 0.02

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# start with a rectangular Figure
fig=plt.figure(1, figsize=(15, 9))

axScatter = plt.axes(rect_scatter)
axHistx = plt.axes(rect_histx)
axHisty = plt.axes(rect_histy)

# no labels
axHistx.xaxis.set_major_formatter(nullfmt)
axHisty.yaxis.set_major_formatter(nullfmt)

vals=axScatter.pcolormesh(X,Y, H, norm=colors.LogNorm(vmin=H.min(),vmax=H.max()), cmap="Blues")
axScatter.tick_params(axis='both', which='major', labelsize=20)
axScatter.set_xticks([0.,0.2,0.4,0.6,0.8])
axScatter.set_xlim(np.sin(np.radians(-6)),1.)
axScatter.set_yticks([2,3,4,5,6,7,8])
axScatter.set_xlabel(r"$\sin(\delta)$", fontsize=24)
axScatter.set_ylabel(r"$\log_{10}(E/\mathrm{GeV})$", fontsize=24)

# now determine nice limits by hand:
#binwidth = 0.25
#xymax = np.max([np.max(np.fabs(x)), np.max(np.fabs(y))])
#lim = (int(xymax/binwidth) + 1) * binwidth
#axScatter.set_xlim((-lim, lim))
#axScatter.set_ylim((-lim, lim))

#bins = np.arange(-lim, lim + binwidth, binwidth)
axHisty.semilogx(Aeff,np.array(setNewEdges(np.concatenate(Y[:,0:1]))))
axHisty.set_xlabel(r"$\log_{10}(\mathrm{A_{eff} (E)}\,[\mathrm{cm}^2])$", fontsize=16,labelpad=5.)
axHisty.set_xticklabels([int(i) for i in np.log10(axHisty.get_xticks())])
axHisty.tick_params(axis='both', which='major', labelsize=14,pad=5.)


axHistx.tick_params(axis='both', which='major', labelsize=14, pad=5.)
axHistx.set_ylabel(r"$\log_{10}(\mathrm{A_{eff} (\delta)}\,[\mathrm{cm}^2])$", fontsize=16,labelpad=5.)
axHistx.semilogy(np.array(setNewEdges(X[0])),np.array(Aeff_theta))
axHistx.set_yticklabels([int(i) for i in np.log10(axHistx.get_yticks())])
axHistx.set_xlim(np.sin(np.radians(-6)),1.)

#cbaxes = plt.axes([0.8, 0.1, 0.03, 0.8]) 
cb = fig.colorbar(vals)  
cb.ax.tick_params(labelsize=16, pad=5) 
cb.ax.set_ylabel(r"$\log_{10}(\mathrm{A_{eff} (E,\delta)}\,[\mathrm{cm}^2])$",fontsize=16, labelpad=-0.1)
#cb.ax.set_yticklabels([int(i) for i in np.log10(cb.ax.get_yticks())])
fig.savefig(SAVE_DIRECTORY+"/test.png", dpi=300)
plt.show()

In [41]:
from scipy.interpolate import interp1d
x=np.degrees(np.arcsin(np.linspace(np.sin(np.radians(-6)),1.,num=len(Aeff_theta))))
Aeff_theta_interpolated = interp1d(x, Aeff_theta)

In [51]:
# Effective area versus the declination angle:

x_new=np.degrees(np.arcsin(np.linspace(np.sin(np.radians(-6)),1.,num=1000*len(Aeff_theta))))
plt.scatter(x,Aeff_theta )
#plt.scatter(x_new,Aeff_theta_interpolated(x_new))
#plt.yscale('log', basey = 10)
plt.xlabel("$\\theta \,[^\circ]$", fontsize=34)
plt.ylabel(r"$A_{\mathsf{eff}} (\theta)\,[\mathsf{cm}^2]$", fontsize=34)
plt.savefig(SAVE_DIRECTORY+"A_eff_scatter.png", dpi=300)
plt.show()

In [49]:
#ready code for getEffAreaVsDec() used in functions.py :
datapath="/net/scratch_icecube4/user/glauch/DATA/"
DETECTOR=LoadDETECTOR("1111")
index=2.13 # assumed spectral index

Aeff_res=plot_aeff_vs_dec_energy(simulationLoad(DETECTOR[1], datapath, 2.13)[0], logE_range=(2,9), sinDec_range=(np.sin(np.radians(-6)),1), bins=[40,40], title=None, savepath=None)

X,Y=np.meshgrid(Aeff_res[2],Aeff_res[1])
H=Aeff_res[0]

Aeff= np.sum(H, axis=1)*(X[0][1]-X[0][0])
deltaE=np.array([10**Aeff_res[1][i+1]-10**Aeff_res[1][i] for i in range(len(Aeff_res[2])-1)])
E=np.array([(10**Aeff_res[1][i+1]+10**Aeff_res[1][i])/2 for i in range(len(Aeff_res[2])-1)])
Aeff_theta=[np.sum((E**(-index))*np.concatenate(H[:,i:i+1])*deltaE)/np.sum((E**(-index))*deltaE) for i in range(len(X)-1)]

x=np.degrees(np.arcsin(np.linspace(np.sin(np.radians(-6)),1.,num=len(Aeff_theta))))
Aeff_theta_interpolated = interp1d(x, Aeff_theta)

/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_11 the total uptime is 29556432.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2011_10602_11077_11191_corrected_ow.npy
EBinWidth 1384260 MC  length 1384260
